# 統計検定


## はじめに

科学や工学の分野では，ある手法や条件が他と比べて優れているかどうかを評価することが重要です．例えば，機械学習やデータ科学分野では，新しく開発した方法が既存の手法よりも良い結果を示すことを証明するために，性能比較が必要です．また，医薬品の開発の現場では，新たに開発した医薬品が効果的であるかの検証が必要です．しかし，多くの研究では，最も良い結果（いわゆる「チャンピオンデータ」）を報告することに重点が置かれ，その結果が統計的に有意な差を持つのかどうかについての議論が不十分であることが少なくありません．

科学的な主張を正当化するには，単に数値の優劣を示すだけでなく，その差が統計的に意味のあるものかを検証しなければなりません．例えば，ある手法の予測性能が従来法よりも高かったとしても，その差が単なる偶然によるものなのか，あるいは本当に再現性のある有意なものなのかを慎重に検討する必要があります．このような判断を適切に行うためには，統計検定という手法が必要不可欠です．

この教材では，統計検定の基本的な概念からはじめ，t検定や多重比較検定を含む代表的な手法を紹介します．これらの手法を理解し活用することで，得られたデータの差が意味のあるものかどうかを客観的に評価できるようになります．科学的な議論の質を高め，より信頼性のある結論を導くために，統計検定の知識を学びます．

## 統計分布について

### データの統計量

### ヒストグラム

### 統計分布

## 統計検定について

### 基本概念

### 仮説検定

### 優位水準とp値

## 集団の平均値の検定

### 基本概念

### 対応の有無

### スチューデントのt検定

### ウェルチのt検定とz検定

## 様々な統計検定法

### パラメトリック

### ノンパラメトリック検定

### 統計検定の繰り返し

A，B，Cというの3群のサンプルデータがあるとき，これらの統計検定つまり，AB間，BC間，CA間の統計検定にt検定を繰り返し利用してはいけません．これらの3組の検定を2群間における3回のt検定で行う場合，少なくともひとつの組み合わせに差が出る確率 $p$ は以下の式で計算されます．

$
p= 1 - p_\phi
$

ここで，$p_\phi$ はすべての組み合わせで差が出ない確率です．今のようにA，B，Cという3群を考える場合，この $p_\phi$ は以下のように計算されます．

$
p_\phi=p_ap_bp_c
$

ここで $p_a$，$p_b$，$p_c$ はそれぞれ，AB間で差が出ない確率，BC間で差が出ない確率，CA間で差が出ない確率です．

これに基づいて $p$ を計算すると，この値は各サンプルデータ間で差が出る確率より大きな数になります．例えば，各サンプルデータ間で差が出る確率を$0.01$とした場合，$p$ は以下のように計算されます．

$
1 - (1 - 0.01) * (1 - 0.01) * (1 - 0.01) = 0.029701
$

これは，各サンプルデータ間で差が出る確率として設定した0.01よりはるかに大きい値です．

$\alpha$ を群間で差が出る確率とおいたとき，少なくともひとつの組み合わせに差が出る確率は以下のようにあらわされます．

$
1 - (1 - \alpha)^n
$

この値は $\alpha$ を $0.05$ とし，$n$ を $10$ として計算すると $0.4013$ となります．これは，群間で差が出る確率がたとえ5%であったとしても検定を10回繰り返すと少なくともそれらの組み合わせて差が出る確率は約40%に達するということを意味します．統計検定を繰り返すと期待以上に有意な差が生じてしまうことになるのです．

### 多重検定の補正

上記のように，複数回のt検定を繰り返すことで少なくとも1回は偶然に有意な差が出てしまう確率（多重比較による第一種の過誤）が増大してしまいます．これを防ぐために，多重比較の影響を考慮したボンフェローニ補正（Bonferroni correction）が用いられます．

ボンフェローニ補正では，元の有意水準 $\alpha$ を比較の回数 $n$ で割ることで各比較における有意水準を厳しく設定します．具体的には，新しい有意水準 $\alpha'$ を次のように計算します．

$
\alpha'=\displaystyle\frac{\alpha}{n}
$

これによって，複数回の検定を行っても全体としての第一種の過誤（偽陽性）の確率を制御する子tができます．

例えば、もともとの有意水準 $\alpha=05$ で3回のt検定（AB間、BC間、CA間）を行う場合，補正後の有意水準は以下のようになります．

$
\alpha'=\displaystyle\frac{0.05}{3}=0.0167
$

つまり，それぞれのt検定のp値が0.0167より小さくないと有意な差があるとは判断しません．ボンフェローニ補正は以下のように計算します．

In [ ]:
#!/usr/bin/env python3
import numpy as np
import scipy.stats as stats
import pandas as pd
from itertools import combinations

np.random.seed(0)

def main():
    # ダミーデータの作成（3群: A, B, C）
    A = np.random.normal(loc=50, scale=10, size=30)
    B = np.random.normal(loc=55, scale=10, size=30)
    C = np.random.normal(loc=60, scale=10, size=30)

    # すべての組み合わせ（AB, BC, CA）でt検定
    groups = {'A': A, 'B': B, 'C': C}
    pairs = list(combinations(groups.keys(), 2))  # ('A', 'B'), ('A', 'C'), ('B', 'C')

    # 有意水準
    alpha = 0.05
    n_tests = len(pairs)  # 検定回数
    alpha_corrected = alpha / n_tests  # ボンフェローニ補正後の有意水準

    # 検定の実施
    results = []
    for g1, g2 in pairs:
        t_stat, p_value = stats.ttest_ind(groups[g1], groups[g2])
        significant = p_value < alpha_corrected
        results.append((g1, g2, p_value, significant))

    df_results = pd.DataFrame(results, columns=["Group 1", "Group 2", "p-value", "Significant (Bonferroni)"])
    print(df_results.to_string(index=False))

if __name__ == "__main__":
    main()

### アドホックな多重検定法

ボンフェローニ補正は複数の検定を行う際に第一種過誤の確率を厳しく抑制するために用いられますが，その分非常に保守的で，有意差を見逃すリスクが高まります．この問題を緩和するために，より柔軟で検出力の高いアドホックな多重検定法がいくつか提案されています．ここでは，代表的な4つの方法を紹介します．

| 方法名 | 適用場面 | 前提条件 |
|---|---|---|
| ダネット検定 | 対照群と複数の処置群の比較 | パラメトリック |
| チューキー・クレーマー法 | すべての群間のペアワイズ比較 | パラメトリック |
| スティール・ドゥワス法 | 対照群と複数の処置群の比較 | ノンパラメトリック |
| ステールの方法 | すべての群間のペアワイズ比較 | ノンパラメトリック |

ダネット検定は複数の処置群と対照群との比較に特化した方法です．対照群との比較のみを行うため，不必要なペアワイズ比較を省略でき，多重比較による誤検出のリスクを効果的に軽減できます．

チューキー・クレーマー法はすべての群間でペアワイズ比較を行う場合に適用される手法です．各比較における統計量はt分布に基づいて計算され，ボンフェローニ補正よりも検出力に優れます．

スティール・ドゥワス法はノンパラメトリックな多重比較法です．パラメトリックのダネット検定と同じく対照群と処置群の比較を行います．各群の順位情報を用いて統計量を算出します．

ステールの方法はノンパラメトリックな枠組みで群間のペアワイズ比較をするための方法です．これも計算は順位情報に基づくため，正規性の仮定が満たされない場合に有用であり，外れ値の影響を受けにくい特徴を持ちます．

これらの方法はどれもボンフェローニ補正と比べると保守性が低く，検出力を向上させる効果が期待できます．手法によって適用条件や前提が異なるため，解析の目的やデータの性質に応じて適切な方法を選択することが重要でしょう．